# **This sbs array_dict with pickle v1**

# **00_setup.py** | *Refences & Imports, def_fnctna, glossary, config flags*

In [ ]:
dir()

## References & Help

**Setup the labtop as a clone of my desktop.**
1. ***Preparing the laptob and the use git to sychrnise***
2. *https://copilot.microsoft.com/shares/q3aDrtDkNPVtV5LMG3eYy*
3. *https://copilot.microsoft.com/shares/rpf9DEopemokLhaR4bXsM*

**Modular Workbook for Array Dictionary Implementation**
1. ***Help setting up the structure of a array_dict system and workbook***
2. *https://copilot.microsoft.com/shares/pages/3HV5tomNJNNZi9VReAwN9*

**Cross-Platform File Path Handling in Python**
1. ***solving problems opening files in window, WWSL  and python***
2. *https://copilot.microsoft.com/shares/pages/txcgXXLkjC8zGKTwGiCd1*

**Using Pathlib to Build File Paths in Python**
1. ***Creatung paths in wsl is tricky using """Pathlib"""to Build File Paths helps***
2. *https://copilot.microsoft.com/shares/pages/SmGr7Zt8ZDNHvbidFGVZy*

**WSL-Compatible File Path Handling in Python**
1. ***variable to represent a file path in WSL***
2. *https://copilot.microsoft.com/shares/pages/bdtSWCe2bNJPvKZSrvDXo*

**Python Pickle Module Guide**
1. ***explaination of Pickel reading and writing***
2. *https://copilot.microsoft.com/shares/pages/qMbW8PsnCpjM1MW2EGH1z*

**Modular Batch Save/Load Utility for array_dict**
1. ***loading many objects to pickle***
2. *https://copilot.microsoft.com/shares/pages/TRT16QCyupBTnq8JwMuSU*

**Modular Pickle Utility Design**
1. ***def functions to read and save Pickel***
2. *https://copilot.microsoft.com/shares/pages/aeBducMjy6ZCcBzYBUW3h*

**Understanding Python Raw Strings**
1. ***r"" is a an operater that instructs how to read the str it doew not change the str r"xl_ada.csv" and "xl_ada.csv" are identical once created—because there are no backslashes to escape.***
2. *Understanding Python Raw Strings*

**CSV Editing and Saving Workflow**
1. ***This loads the dat_col dict for .csv file and then after editing replace the updated version into same file***
2. *https://copilot.microsoft.com/shares/pages/QabFDtDG8Utc8EFMgzzyz*

**Ordered Dictionary with 'dvt' First in Python**
1. ***This reads a 2D array with headers from .csv file and makes a std dat_obj like [ad_sn]***
2. *https://copilot.microsoft.com/shares/pages/3iojVoSLAcCggGcWtxUmM*

**Optimizing Data Types for Memory Efficiency**
1. ***This should be used before I start expanding***
2. *https://copilot.microsoft.com/shares/pages/wGUNBVFLcxarPtGE1mwnP*

**Using try: on the index col**
1. ***Try: ia valuable tool used with index method of panda***
2. *https://copilot.microsoft.com/shares/pages/iqre2v246hhjkauLu1CGG*

**Calculatioing names of load and save results**
1. ***Complicated way to calculate namees of functions results***
2. *https://copilot.microsoft.com/shares/pages/kzs5ZkCweSW2g8TEVWwBY*

**Function Registry**
1. ***Build a registry to hold and call functions***
2. *https://copilot.microsoft.com/shares/VAT6QDNijDghzbWkei5fV*

**🧩 Step-by-Step: Assign "CoQ10" as a ResourceLoader Instance"**
1. ***create an instance of a certain class***
2. *https://copilot.microsoft.com/shares/PfLQMHCkQZXy9RoHAerAZ*

**Produce an instance of that class for each dat_col**
1. ***Once you define a class that includes attributes like name and normal_dose, and then apply it to each item in your dat_col, running __init__ will produce an instance of that class for each item..***
2. *https://copilot.microsoft.com/shares/VAT6QDNijDghzbWkei5fV*

**List of copilot pages on Classes**
1. ***buils class system links in reverse order***
2. *https://copilot.microsoft.com/shares/YZwoHgYtMtPo9mySMxHQL* ***Inheritance-Based Classification Scaffold***
3. *https://copilot.microsoft.com/shares/dgoHQbZaYRVMaC7teBiqX*
4. *https://copilot.microsoft.com/shares/qASaQdp29maniC2mDnY4h*
5. *https://copilot.microsoft.com/shares/vnTrfzUidrguteQgZuSNU*
6. *https://copilot.microsoft.com/shares/TWNAfv6ubx8z2bAiLA8oX*
7. *https://copilot.microsoft.com/shares/reShWjrM9yt9Hgu1HNVk7*
8. *https://copilot.microsoft.com/shares/HAtG3uKHezAPpYNWdwZBv*
9. *https://copilot.microsoft.com/shares/MApX5LfmaD2ipvn65PihY* ***Why Dual Instancing of dat_col Is Safe and Smart***
10. *https://copilot.microsoft.com/shares/HSFhgptLeXqQe8MuDp31k* ***When i commt to git and shut down, will I be able to use the class operaters to generate instances when I restart.***
11. *https://copilot.microsoft.com/shares/i72SNPPtQBbav6fsu4cT4* ***Here's a restart-ready bootstrap scaffold that wires together your canonical loader, filtering class, and diagnostic overlays. It’s modular, traceable, and operator-friendly—just how you like it.***
12. *https://copilot.microsoft.com/shares/JXNEwRKMDoigkwLMymtbd* ***When i commt to git and shut down, will I be able to use the class operaters to generate instances when I restart.***
13.  next

**JupyterLab is a great sandbox for building and visualizing hierarchical structures like ancestral trees, signal pathways, and organizational charts**
1. ***Build a strategy for diagrams built in jupyterlab***
2. *https://copilot.microsoft.com/shares/xGFuAUQg6yJZ62g3HayrQ* **all options for diagraming in JL**
3. https://copilot.microsoft.com/shares/DvFDux4kBCxokgYujPS7x   **pick final approach for diagrams inJL**
4. *https://copilot.microsoft.com/shares/MjejKcx6aTazHEp3HK4JL* ***explanation and scaffold***
5. * https://copilot.microsoft.com/shares/pWonoyBkJ3DYQacVespbd* ***converts the diagram loader to a function*

**Function Registry**
1. ***Build a registry to hold and call functions***
2. *https://copilot.microsoft.com/shares/VAT6QDNijDghzbWkei5fV*

**Function Registry**
1. ***Build a registry to hold and call functions***
2. *https://copilot.microsoft.com/shares/VAT6QDNijDghzbWkei5fV*

**📱 iPhone Input Strategies for dat_col Dicts**
1. ***use cell phones to enter daily data***
2. *https://copilot.microsoft.com/shares/wsUJ7hi4LfdANpwz4njf3*  ***selecting strategy***
3. https://copilot.microsoft.com/shares/xCioyq4yenpv714NtzLeC * ***designing in Jupyter labs***

## Imports,glossary, Def_function_utilities, config flags

### Imports

In [3]:
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from collections import OrderedDict
base_path = os.path.expanduser("~")  # Gets /home/bhuns in WSL all measured from here
REPO = ""
#data = " \" + "_data"
print("REPO = ",REPO)
print("base_path = ",base_path)
print("REPO\data = ",REPO + "\data")
print("sys.executable = ",sys.executable)

REPO =  
base_path =  /home/bhuns
REPO\data =  \data
sys.executable =  /home/bhuns/miniconda3/envs/scu/bin/python


### Glossary

xl lists def functions  bnms  ......

#### List of bnm [base_names] for arrays that need to be imported and formatted

### Def_function_utilities

#### array load/save policy

1. Full Round trip def function
2. for csv storage
3. from a **"bnm"** [Basic data class Name]
4. and a **adsn** [Array Dict Serial Number]

#### Utilities >>> contents of directories

In [17]:
# if name is in directory
ad_an for name, ad_an in globals().items() if isinstance(val, pd.DataFrame)]


SyntaxError: unmatched ']' (1746734617.py, line 2)

In [4]:
# all namees in directory
dir()

['In',
 'OrderedDict',
 'Out',
 'Path',
 'REPO',
 '_',
 '_1',
 '_3',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__session__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'base_path',
 'csv',
 'datetime',
 'exit',
 'get_ipython',
 'np',
 'open',
 'os',
 'pd',
 'pickle',
 'quit',
 'tk']

#### defs/calls >>> 1 load_pickle(bmn)

In [5]:
def load_pickle(bmn):
    """Load object from Pickle file."""
    path_pkl = Path("data") / f"{bnm}.pkl"
    str(path_pkl)                           # change to string
    path = Path(path_pkl)
    print(bnm, ">>> is pickled to >>> :",path_pkl)
    with path.open('rb') as f:
        return pickle.load(f)


In [6]:
bnm = "ad_sn"            # this is file where loads in data
load_pickle(bnm)
                                                             #Test

ad_sn >>> is pickled to >>> : data/ad_sn.pkl


'ad_sn'

#### defs/calls >>> 2 save_pickle(bmn)

In [8]:
def save_pickle(obj, bnm):
    """Save object to Pickle file."""
    path_pkl = Path("data") / f"{bnm}.pkl"
    print (obj, "is recorded to pickle file > ",path_pkl)
    with path_pkl.open('wb') as f:
        pickle.dump(obj, f)


In [9]:
#obj

In [10]:
bnm

'ad_sn'

#### defs/calls >>> 3a load_csv_to_df(bnm)

In [11]:
def load_csv_to_df(bnm):
    path_csv = Path(f"{bnm}.csv")
    #path_csv

#### defs/calls >>> 3b load_csv_to_dtv_dict(bnm)

In [19]:
def load_csv_to_dict(bnm):
C    """
    Load CSV into DataFrame and convert to ordered dict with 'dvt' first.
    Sets 'dvt' as string index for direct access.
    """
    path_csv = Path(f"{bnm}.csv")
    df = pd.read_csv(path_csv)
    print(df)
    # Ensure 'dvt' is string before setting index
    df['dvt'] = df['dvt'].astype(str)
    df.set_index('dvt', inplace=True)

    # Build ordered dictionary with 'dvt' as first key
    obj = OrderedDict()
    obj['dvt'] = df.index.tolist()

    for col in df.columns:
        #obj[col] = df[col].tolist()  # converts to panda series
        obj[col] = df[col].to_numpy() # converts to numpy
    print(obj)
    return obj


IndentationError: expected an indented block after function definition on line 1 (524616303.py, line 2)

In [13]:
#ad_sn

#### defs/calls >>> make an upload to load pkl after xl edit?

#### defs/calls >>>  4 save_csv(bnm, obj)

In [14]:
# save Dictionary to csv in REPO\
# Note it converts to df and saves!!!!!!!
def save_csv(bnm, obj):
    """
    Reconstruct DataFrame from edited dictionary and overwrite original CSV.
    Assumes 'dvt' is present as the first key in obj.
    """
    # Rebuild DataFrame from dictionary
    df_out = pd.DataFrame(obj)

    # Ensure column order starts with 'dvt'
    Lcols = ['dvt'] + [col for col in df_out.columns if col != 'dvt']
    df_out = df_out[cols]

    # Write back to original file
    path_csv = f"{bnm}.csv"
    df_out.to_csv(path_csv, index=False)
    #print("saved to file > ",path_csv)
    


In [15]:
bnm = "ad_sn"
obj = array_dict[bnm]  # assuming array_dict is your metadata-rich dictionary

save_csv(bnm, obj)



NameError: name 'array_dict' is not defined

##### calls 

In [ ]:
# Load array from f"{bnm}.csv" in REPO  to a array_dict of name "bnm" in memory
bnm ="ad_sn"                                # copy and paste "bnm test " to be name of final result in memory
#df, ad_sn = load_csv_to_dict(bnm)  # Load original
#df,ad_sn  
ad_sn = load_csv_to_dict(bnm)  # Load original# test only

In [ ]:
ad_sn

In [ ]:
#Set a value in dct
obj = ad_sn
dvt_key = 45863
col_name = "s002"
value = 12
set_col_value(obj, dvt_key, col_name, value)

In [ ]:
ad_sn

In [ ]:
# Call to  # def to save pkl from """bnm""""" [SELECTS THE PATH AND FILE NAME]
bnm ="ad_sn"
(load_pickle(bnm))


In [ ]:
obj


In [ ]:
bnm = "name"
name = load_pickle(bnm)}

##### defs/calls >>> set_col_value(obj, dvt_key, col_name, value)

In [ ]:
# set a value in a specific item
def set_col_value(obj, dvt_key, col_name, value):
    try:
        idx = obj['dvt'].index(str(dvt_key))
        obj[col_name][idx] = value
        print(idx)
    except ValueError:
        print(f"dvt {dvt_key} not found.")
    except KeyError:
        print(f"Column {col_name} not found.")


#### pkl load/save  

In [ ]:
# from copilot page
def save_pickle(obj, path):
    """Save object to Pickle file."""
    path = Path(path)
    with path.open('wb') as f:
        pickle.dump(obj, f)

def load_pickle(path):
    """Load object from Pickle file."""
    path = Path(path)
    with path.open('rb') as f:
        return pickle.load(f)

In [ ]:
# pickle load call
path = "data/bnm"

In [ ]:
# pickle save call
path = "data/bnm"
obj = "bnm"

In [ ]:
# pkl load/save
def save_pkl(bnm, array_dict):

In [ ]:

    path_pkl = Path("REPO/data") / f"{bnm}.pkl"

    # Save updated dictionary back to Pickle
    with open(path_pkl, 'wb') as f:
        pickle.dump(array_dict, f)

In [ ]:
    """
    Load Pickle file, apply edits to array_dict, and overwrite original file.
    Assumes file is stored in REPO/data/{bnm}.pkl
    """

In [ ]:
# Load original Pickle
with open("REPO/data/my_data.pkl", 'rb') as f:
    array_dict = pickle.load(f)

# ... perform edits to array_dict ...

# Save back to same Pickle file
load_edit_save_pkl("my_data", array_dict)


In [ ]:
bnm = "ad_sn"
array_dict = "ad_sn"
load_edit_save_pkl(bnm, array_dict)

# **02_array_dict_builder.py**   |  *Transform arrays into array_dict with metadata*
_ ***Script Update for Data Dictionary Creation** link *https://copilot.microsoft.com/shares/pages/jLLrDDP9aZqQjatwkyFfr*


In [ ]:
# Read and isolate the raw headers from "xl_ad_sn_hdr,csl"
# Load CSV


csv_path = os.path.join(os.path.expanduser("~"), "JL_1/data/xl_ad_sn.csv")
df = pd.read_csv(csv_path)

# Inspect raw headers
raw_headers = df.columns.tolist()
#print("Raw headers:", raw_headers)

In [ ]:
# ✅ Full Setup >> headers >> dtv_index >> array_dict
csv_path = os.path.join(os.path.expanduser("~"), "JL_1/data/xl_ad_sn.csv")
df = pd.read_csv(csv_path)         # read array from "csv_path"

# Extract headers and data
headers = df.columns.tolist()
xl_ad_sn = df.to_numpy()

# Locate the index of the 'dtv' column
dtv_index = headers.index('dtv')

# Build array_dict
array_dict = {}

for row in xl_ad_sn:
    dtv_key = row[dtv_index]
    raw_values = np.delete(row, dtv_index)

    array_dict[dtv_key] = {
        'raw_values': raw_values.tolist(),
        'flags': {},
        'attributes': {
            'features': [h for i, h in enumerate(headers) if i != dtv_index]
        },
        'classification': {}
    }


In [ ]:
# 🧪 Verify the construction

for dtv, entry in list(array_dict.items())[:3]:
    print(f"DTV: {dtv}")
    print("Raw:", entry['raw_values'])
    print("Features:", entry['attributes']['features'])
    print("-" * 40)


In [ ]:
# Step 3: Continue Building array_dict
xl_ad_sn = df.to_numpy()

array_dict = {}
for row in xl_ad_sn:
    dtv_key = row[dtv_index]
    raw_values = np.delete(row, dtv_index)

    array_dict[dtv_key] = {
        'raw_values': raw_values.tolist(),
        'flags': {},
        'attributes': {
            'features': [h for i, h in enumerate(raw_headers) if i != dtv_index]
        },
        'classification': {}
    }



In [ ]:
for dvt, entry in list(array_dict.items())[:3]:
    print(f"DVT: {dvt}")
    print("Raw:", entry['raw_values'])
    print("Features:", entry['attributes']['features'])
    print("-" * 40)

# 03 Setting the classification of "dat_col"s with attribts stored in files.

## Introduction

### **Metadata Injection Function in Python**
_
*https://copilot.microsoft.com/shares/pages/LpNfiWnk3ndWRko9EnfxS*

_Question *I would like like to have function that would accept a "dat_col_sn" and and a "attribut_sn" and read the "value" in that item in the file "attrbts_sn_x_dat_col_sn" and set that "attribute" in that "dat_col" to that "value". If the class & attrbute did not exist, it would create one. Js that possible.*

Absolutely, Bill—that’s not only possible, it’s a great example of modular, operator-driven metadata injection. You’re describing a function that:

Accepts a dat_col_sn (e.g. 's003') and an attribute_sn (e.g. 'a001')

Looks up the value at that coordinate in a matrix file (e.g. CSV or Excel)

Injects that value into the corresponding array_dict[dat_col_sn]['attributes'][attribute_sn]

Creates the attribute if it doesn’t exist

Let’s scaffold that cleanly.

## 🧠 Function: inject_attribute_from_matrix

### 🔹 Assumptions

1. The matrix file is named "attrbts_sn_x_dat_col_sn.csv" ..................ok  [matrix_path]
2. Rows = attribute_sn (e.g. 'a000', 'a001')................................ok
3. Columns = dat_col_sn (e.g. 's000', 's001')...............................ok
4. You’ve already built array_dict keyed by dat_col_sn.........need to load ##
"\\wsl.localhost\Ubuntu\home\bhuns\JL_1\data\attrbts_sn_x_dat_col_sn.csv"
r"data\attrbts_sn_x_dat_col_sn.csv"
"attrbts_sn_x_dat_col_sn.csv"

In [ ]:
filename = "attrbts_sn_x_dat_col_sn.csv"
df = read_wsl_csv(filename)
df

### 🛠️ Implementation

#### inject_attribute_from_matrix(array_dict, dat_col_sn, attribute_sn, matrix_path):

In [ ]:
# inject_attribute_from_matrix(array_dict, dat_col_sn, attribute_sn, matrix_path):

def inject_attribute_from_matrix(array_dict, dat_col_sn, attribute_sn, matrix_path):
    # Load matrix
    df = pd.read_csv(matrix_path, index_col=0)  # Assumes first column is attribute_sn

    # Validate existence
    if attribute_sn not in df.index:
        raise ValueError(f"Attribute '{attribute_sn}' not found in matrix rows.")
    if dat_col_sn not in df.columns:
        raise ValueError(f"Data column '{dat_col_sn}' not found in matrix columns.")

    # Extract value
    value = df.at[attribute_sn, dat_col_sn]

    # Inject into array_dict
    if dat_col_sn not in array_dict:
        raise KeyError(f"Data column '{dat_col_sn}' not found in array_dict.")

    entry = array_dict[dat_col_sn]
    if 'attributes' not in entry:
        entry['attributes'] = {}

    entry['attributes'][attribute_sn] = value

    return value  # Optional: return for confirmation


In [ ]:
#🧪 Example Usage
matrix_path = os.path.join(os.path.expanduser("~"), "JL_1/data/attrbts_sn_x_dat_col_sn.csv")
value = inject_attribute_from_matrix(array_dict, 's003', 'a001', matrix_path)

print(f"Injected value: {value}")
print(array_dict['s003']['attributes']['a001'])


#### 🔍 Optional Enhancements

Add logging or provenance tags

Support batch injection for all attributes or all columns

Validate types or ranges before injection

### next###

#### next####

### next###

#### next####

### next###

#### next####

### next###

#### next####

# **05_storage_utils.py**  |  *Save/load using Pickle, Excel, HDF5*

# **04_gui_selector.py**  |  *Operator-driven GUI for selecting and grouping entries*

# **0?_classification_registry.py**  |  *Define classification args, flags, and validation logic*
_***Wrking but funct

**Classification Registry Implementation** 
_ *https://copilot.microsoft.com/shares/pages/P91viiY6QzR3Mp4WUnbFf*

## 🧠 Classification Registry Scaffold (classification_registry.py)

### Perfect, Bill. You’ve got a classification matrix scaffolded—rows as attribute_sn (e.g. a000, a001, ...) and columns as dat_col.sn (e.g. s000, s001, ...). 
1. That’s a great foundation for a modular classification registry.
2. Let’s build a separate file—say, classification_registry.py—that can:
3. Load and validate the matrix
4. Apply classification logic per attribute or per column
5. Track provenance and confidence
6. Return structured results for embedding into array_dict

In [ ]:
#🔹 1. Load the Matrix

def load_classification_matrix(path):
    df = pd.read_csv(path)  # or pd.read_excel
    df.index.name = 'attribute_sn'
    return df


In [ ]:
# 🔹 2. Define Classification Methods
def threshold_classifier(row, threshold=0.5):
    score = row.mean()
    group = 'A' if score > threshold else 'B'
    confidence = round(abs(score - threshold), 3)
    return group, confidence

def zscore_classifier(row):
    z = (row - row.mean()) / row.std()
    group = 'High' if z.mean() > 1 else 'Low'
    confidence = round(z.std(), 3)
    return group, confidence


In [ ]:
# 🔹 3. Registry Dictionary
classification_registry = {
    'threshold': threshold_classifier,
    'zscore': zscore_classifier
}


In [ ]:
# 🔹 4. Apply Classification
def classify_matrix(df, method='threshold'):
    results = {}
    classifier = classification_registry[method]

    for attr_sn, row in df.iterrows():
        group, confidence = classifier(row)
        results[attr_sn] = {
            'group': group,
            'confidence': confidence,
            'method': method
        }
    return results


In [ ]:
# 🔹 5. Embed into array_dict
def embed_classification(array_dict, classification_results):
    for dvt_key, result in classification_results.items():
        if dvt_key in array_dict:
            array_dict[dvt_key]['classification'] = result


In [ ]:
# 🧪 Optional Diagnostic
def preview_classification(results, n=5):
    for k in list(results.keys())[:n]:
        print(f"{k}: {results[k]}")


# **06_diagnostics.py**  |  *Overlay tools for inspecting metadata and classification*

# **07_tests.py**  |  *Reproducible test cases and versioning checks*